In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib
from ipywidgets import interact

/home/wbeckner/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# API

send - type/treatment, GSM, full width line 11, full width line 13; receive neck-in and confidence level

In [2]:
def schedule_api(treat, gsm, width11, width13, slitwidth):
    knn = joblib.load("../data/slitter_knife_loss/{}.sav".format(treat))
    pred = knn.predict([[gsm, width11, width13, slitwidth]])
    pred = round(pred[0])
    return """{}""".format(pred)

In [3]:
schedule_api("SSS Untreated", 9, 4325, 4365, 400)

/home/wbeckner/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KNeighborsRegressor from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


'5.0'

# Train

In [ ]:
sheets = ['SSS Untreated', 'SMS Untreated', 'SMS Treated-ZT', 
          'SSS Treated-ZT', 'SSS Kami Untreated', 'SSS Kami Treated-ZT',
          'SMS Kami Untreated', 'SMS Kami Treated-ZT']
sheets2 = ['SSS Bico',
          'SSS Bico Treated', 'SSS Vistamax', 'SSS P&G irgatec', 
          'SSS SB2 Prime']
for sheet in sheets:
    df = pd.read_excel('../data/slitter_knife_loss/WebWidthOptimizer.xlsx', 
                  sheet_name=sheet)
#     sns.heatmap(df.iloc[:,3:])
    ls = []
    for row in df.index:
        for col in df.columns[3:]:
            stuff = list(np.append(df.iloc[row,:3].values, col))
            stuff.append(df.iloc[row][col])
            ls.append(stuff)
    data = np.array(ls)
    print(data.shape)
    x = data[:,:-1]
    y = data[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    knn = KNeighborsRegressor(n_neighbors=2)
    knn.fit(X_train, y_train)
    print('80/20 train/test, train score: {:.4f}'.format(knn.score(X_train, y_train)))
    print('80/20 train/test, test score: {:.4f}'.format(knn.score(X_test, y_test)))
    knn = KNeighborsRegressor(n_neighbors=2)
    knn.fit(x,y)
    print('Full dataset score: {:.4f}'.format(knn.score(x, y)), end='\n\n')
#     joblib.dump(knn, "{}.sav".format(sheet))

In [ ]:
sheets = ['SSS Untreated', 'SMS Untreated', 'SMS Treated-ZT', 
          'SSS Treated-ZT', 'SSS Kami Untreated', 'SSS Kami Treated-ZT',
          'SMS Kami Untreated', 'SMS Kami Treated-ZT']   
def plot_heat(sheet=sheets):
    df = pd.read_excel('../data/slitter_knife_loss/WebWidthOptimizer.xlsx', 
                  sheet_name=sheet)
    fig, ax = plt.subplots(1, 1, figsize=(10,10))
    sns.heatmap(df.iloc[:,3:], ax=ax)
    ax.set_title(f'{sheet}')

In [ ]:
interact(plot_heat)

In [ ]:
print(knn.score(X_train, y_train))
print(knn.score(X_test, y_test))